In [5]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [31]:
data = pd.read_csv('/content/drive/My Drive/BSFR.csv')
data = data[::-1]
data

,Date,Close,Open,High,Low,Vol.,Change %
882,1/2/2022,23.07,22.66,23.12,22.56,661.87K,1.27%
881,1/3/2022,22.66,23.07,23.07,22.54,896.12K,-1.78%
880,1/4/2022,22.32,22.85,22.85,22.25,941.47K,-1.50%
879,1/5/2022,22.42,22.52,22.52,22.32,897.16K,0.45%
878,1/6/2022,22.23,22.42,22.42,22.18,444.16K,-0.85%
...,...,...,...,...,...,...,...
4,7/16/2025,17.44,17.60,17.66,17.43,2.33M,-0.51%
3,7/17/2025,17.46,17.53,17.59,17.34,1.40M,0.11%
2,7/20/2025,17.46,17.58,17.63,17.33,886.83K,0.00%
1,7/21/2025,17.63,17.60,17.63,17.40,1.69M,0.97%


In [32]:
# Find the lowest price for the stock over the last week
data["prv_low_1"] = data["Low"].shift(1)
data["prv_low_2"] = data["Low"].shift(2)
data["prv_low_3"] = data["Low"].shift(3)
data["prv_low_4"] = data["Low"].shift(4)
data["prv_low_5"] = data["Low"].shift(5)
data["Low_Last_5d"] = data[["prv_low_1", "prv_low_2","prv_low_3","prv_low_4","prv_low_5"]].values.min(1)
data = data.drop(["prv_low_1", "prv_low_2", "prv_low_3", "prv_low_4", "prv_low_5"], axis=1)
data.tail(2)

,Date,Close,Open,High,Low,Vol.,Change %,Low_Last_5d
1,7/21/2025,17.63,17.60,17.63,17.40,1.69M,0.97%,17.33
0,7/22/2025,17.26,17.98,18.01,17.24,4.24M,-2.10%,17.33


In [33]:
# Create boolean variable marking whether stock dropped at least six percent in the last week (five days)
data["six_percent_of_close_price"] = .94*data["Close"]
data["Close"] = 1*data["Close"]
data["six_percent_of_close_price_within_last_five_days"] = np.where((data['Low_Last_5d'] <= data['six_percent_of_close_price']), True, False)
#data["six_percent_of_close_price_within_last_five_days"] = np.where((data['six_percent_of_close_price'] <= data['Low_Last_5d']), True, False)

In [34]:
# Create boolean variable marking whether current close price is greater than yesterdays close price
data["Last_Close"] = data["Close"].shift(1)
data["close_diff"] = data["Close"].values.reshape(data.shape[0]) - data["Last_Close"].values.reshape(data.shape[0])
data["up_from_yesterday"] = np.where((data["close_diff"] > 0), True, False)
# Add numerical columns of stocks current close price increased and decrease by one percent for final analysis
data["Close_up1%"] = 1.01*data["Close"]
data["Close_down1%"] = .99*data["Close"]
data.tail(10)

,Date,Close,Open,High,Low,Vol.,Change %,Low_Last_5d,six_percent_of_close_price,six_percent_of_close_price_within_last_five_days,Last_Close,close_diff,up_from_yesterday,Close_up1%,Close_down1%
9,7/9/2025,17.99,18.19,18.19,17.86,989.18K,-0.72%,17.51,16.9106,False,18.12,-0.13,False,18.1699,17.8101
8,7/10/2025,17.95,18.07,18.07,17.78,831.98K,-0.22%,17.58,16.8730,False,17.99,-0.04,False,18.1295,17.7705
7,7/13/2025,17.99,17.95,18.01,17.82,518.48K,0.22%,17.78,16.9106,False,17.95,0.04,True,18.1699,17.8101
6,7/14/2025,17.69,17.90,18.02,17.69,2.10M,-1.67%,17.78,16.6286,False,17.99,-0.30,False,17.8669,17.5131
5,7/15/2025,17.53,17.69,17.76,17.38,2.38M,-0.90%,17.69,16.4782,False,17.69,-0.16,False,17.7053,17.3547
4,7/16/2025,17.44,17.60,17.66,17.43,2.33M,-0.51%,17.38,16.3936,False,17.53,-0.09,False,17.6144,17.2656
3,7/17/2025,17.46,17.53,17.59,17.34,1.40M,0.11%,17.38,16.4124,False,17.44,0.02,True,17.6346,17.2854
2,7/20/2025,17.46,17.58,17.63,17.33,886.83K,0.00%,17.34,16.4124,False,17.46,0.00,False,17.6346,17.2854
1,7/21/2025,17.63,17.60,17.63,17.40,1.69M,0.97%,17.33,16.5722,False,17.46,0.17,True,17.8063,17.4537
0,7/22/2025,17.26,17.98,18.01,17.24,4.24M,-2.10%,17.33,16.2244,False,17.63,-0.37,False,17.4326,17.0874


In [35]:
# Look into date Mohammed says should have been entered (5/29/25)
data["shortsell"] = data["six_percent_of_close_price_within_last_five_days"] & data["up_from_yesterday"]
data['month'] = pd.DatetimeIndex(data['Date']).month
data['year'] = pd.DatetimeIndex(data['Date']).year

In [36]:
data.loc[((data.month==5) & (data.year==2024))].tail(5)

,Date,Close,Open,High,Low,Vol.,Change %,Low_Last_5d,six_percent_of_close_price,six_percent_of_close_price_within_last_five_days,Last_Close,close_diff,up_from_yesterday,Close_up1%,Close_down1%,shortsell,month,year
287,5/26/2024,16.13,16.61,16.73,16.10,1.75M,-2.90%,16.56,15.1622,False,16.61,-0.48,False,16.2913,15.9687,False,5,2024
286,5/27/2024,15.91,16.25,16.25,15.67,3.93M,-1.35%,16.10,14.9554,False,16.13,-0.22,False,16.0691,15.7509,False,5,2024
285,5/28/2024,15.79,16.01,16.03,15.74,1.84M,-0.76%,15.67,14.8426,False,15.91,-0.12,False,15.9479,15.6321,False,5,2024
284,5/29/2024,16.25,15.91,16.54,15.91,4.00M,2.90%,15.67,15.2750,False,15.79,0.46,True,16.4125,16.0875,False,5,2024
283,5/30/2024,16.15,16.25,16.37,16.08,4.18M,-0.59%,15.67,15.1810,False,16.25,-0.10,False,16.3115,15.9885,False,5,2024


In [37]:
data.loc[data.six_percent_of_close_price_within_last_five_days==True]

,Date,Close,Open,High,Low,Vol.,Change %,Low_Last_5d,six_percent_of_close_price,six_percent_of_close_price_within_last_five_days,Last_Close,close_diff,up_from_yesterday,Close_up1%,Close_down1%,shortsell,month,year
862,1/30/2022,25.22,24.83,25.31,24.78,1.57M,1.57%,23.53,23.7068,True,24.83,0.39,True,25.4722,24.9678,True,1,2022
861,1/31/2022,25.22,25.31,25.79,25.22,1.74M,0.00%,23.53,23.7068,True,25.22,0.00,False,25.4722,24.9678,False,1,2022
860,2/1/2022,25.41,25.36,25.75,25.22,640.57K,0.75%,23.53,23.8854,True,25.22,0.19,True,25.6641,25.1559,True,2,2022
840,3/2/2022,25.79,25.55,25.99,25.22,1.02M,0.55%,24.16,24.2426,True,25.65,0.14,True,26.0479,25.5321,True,3,2022
829,3/17/2022,25.70,24.78,25.70,24.69,3.79M,4.09%,24.11,24.1580,True,24.69,1.01,True,25.9570,25.4430,True,3,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,4/24/2025,18.46,18.36,18.58,18.32,3.05M,0.44%,17.10,17.3524,True,18.38,0.08,True,18.6446,18.2754,True,4,2025
58,4/27/2025,18.40,18.48,18.60,18.32,469.83K,-0.33%,17.26,17.2960,True,18.46,-0.06,False,18.5840,18.2160,False,4,2025
18,6/26/2025,17.52,17.40,17.52,17.28,1.85M,0.92%,16.32,16.4688,True,17.36,0.16,True,17.6952,17.3448,True,6,2025
17,6/29/2025,17.80,17.64,17.80,17.48,769.19K,1.60%,16.40,16.7320,True,17.52,0.28,True,17.9780,17.6220,True,6,2025


In [ ]:
data.loc[((data.month==5) & (data.year==2024))].tail(10)

Price,Close,High,Low,Open,Volume,Low_Last_5d,six_percent_of_close_price,six_percent_of_close_price_within_last_five_days,Last_Close,close_diff,up_from_yesterday,Close_up1%,Close_down1%,shortsell,date,month,year
Ticker,1050.SR,1050.SR,1050.SR,1050.SR,1050.SR,,,,,,,,,,,,
Date,,,,,,,,,,,,,,,,,
2024-05-19,16.329464,16.329464,16.329464,16.329464,0,16.019691,15.349696,False,16.329464,0.000000,False,16.492759,16.166169,False,2024-05-19,5,2024
2024-05-20,16.329464,16.329464,16.329464,16.329464,0,16.019691,15.349696,False,16.329464,0.000000,False,16.492759,16.166169,False,2024-05-20,5,2024
2024-05-21,15.864805,16.130325,15.842679,16.130325,1038142,16.019691,14.912917,False,16.329464,-0.464659,False,16.023453,15.706157,False,2024-05-21,5,2024
2024-05-22,15.864805,15.864805,15.864805,15.864805,0,15.842679,14.912917,False,15.864805,0.000000,False,16.023453,15.706157,False,2024-05-22,5,2024
2024-05-23,16.320036,16.320036,16.320036,16.320036,0,15.842679,15.340834,False,15.864805,0.455231,True,16.483236,16.156836,False,2024-05-23,5,2024
2024-05-26,15.227482,15.796521,15.204720,15.682714,1754517,15.842679,14.313833,False,16.320036,-1.092554,False,15.379757,15.075207,False,2024-05-26,5,2024
2024-05-27,15.227482,15.227482,15.227482,15.227482,0,15.204720,14.313833,False,15.227482,0.000000,False,15.379757,15.075207,False,2024-05-27,5,2024
2024-05-28,15.227482,15.227482,15.227482,15.227482,0,15.204720,14.313833,False,15.227482,0.000000,False,15.379757,15.075207,False,2024-05-28,5,2024


In [48]:
data.iloc[0]["Close_up1%"]

np.float64(23.3007)

In [49]:
currently_entered=False
close_u1 = np.nan
close_d1 = np.nan
close = np.nan

entry_prices = []
entry_dates = []

exit_prices = []
exit_dates = []


for i in range(len(data)):
  close = data.iloc[i]["Close"]
  if (currently_entered==False):
    if (data.iloc[i]["six_percent_of_close_price_within_last_five_days"]==True and data.iloc[i]["up_from_yesterday"]==True):
      currently_entered = True
      close_u1 = data.iloc[i]["Close_up1%"]
      close_d1 = data.iloc[i]["Close_down1%"]
      entry_prices.append(close)
      entry_dates.append(data.iloc[i]["Date"])
  else:
    if (close >= close_u1 or close <= close_d1):
      exit_prices.append(close)
      exit_dates.append(data.iloc[i]["Date"])
      currently_entered = False

In [50]:
print(entry_dates)
print(exit_dates)
print()
print(entry_prices)
print(exit_prices)

['1/30/2022', '3/2/2022', '3/17/2022', '3/22/2022', '4/25/2022', '5/29/2022', '6/16/2022', '6/28/2022', '7/18/2022', '7/21/2022', '8/1/2022', '9/29/2022', '10/5/2022', '10/18/2022', '11/24/2022', '11/29/2022', '12/14/2022', '12/28/2022', '3/2/2023', '3/19/2023', '3/21/2023', '4/3/2023', '4/12/2023', '6/13/2023', '7/4/2023', '7/12/2023', '7/16/2023', '10/11/2023', '10/16/2023', '11/1/2023', '11/6/2023', '11/20/2023', '12/14/2023', '12/19/2023', '1/7/2024', '1/29/2024', '6/2/2024', '6/9/2024', '7/18/2024', '12/9/2024', '3/26/2025', '4/8/2025', '4/10/2025', '4/14/2025', '4/23/2025', '6/26/2025', '6/30/2025']
['2/2/2022', '3/3/2022', '3/21/2022', '3/30/2022', '4/26/2022', '5/30/2022', '6/19/2022', '6/30/2022', '7/19/2022', '7/24/2022', '8/7/2022', '10/4/2022', '10/6/2022', '10/23/2022', '11/28/2022', '12/1/2022', '12/15/2022', '1/2/2023', '3/7/2023', '3/20/2023', '3/29/2023', '4/4/2023', '4/17/2023', '6/15/2023', '7/5/2023', '7/13/2023', '7/19/2023', '10/15/2023', '10/22/2023', '11/5/2023'

In [53]:
entry_exit_values = []
dates = []
cumulative_profit  = 0
entryexitframe = pd.DataFrame(columns=["ticker", "entry_date", "entry_price", "exit_date", "exit_price", "profit_percentage"])
for i in range(min(len(entry_prices),len(exit_prices))):
  cumulative_profit += (entry_prices[i]-exit_prices[i])
  entryexitframe = pd.concat([pd.DataFrame([["1050.SR",entry_dates[i],entry_prices[i],exit_dates[i],exit_prices[i],((entry_prices[i]-exit_prices[i])/entry_prices[i])*100]], columns=entryexitframe.columns), entryexitframe], ignore_index=True)

/tmp/ipython-input-53-17905189.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  entryexitframe = pd.concat([pd.DataFrame([["1050.SR",entry_dates[i],entry_prices[i],exit_dates[i],exit_prices[i],((entry_prices[i]-exit_prices[i])/entry_prices[i])*100]], columns=entryexitframe.columns), entryexitframe], ignore_index=True)


In [54]:
entryexitframe[::-1]

,ticker,entry_date,entry_price,exit_date,exit_price,profit_percentage
46,1050.SR,1/30/2022,25.22,2/2/2022,24.59,2.498017
45,1050.SR,3/2/2022,25.79,3/3/2022,25.51,1.085692
44,1050.SR,3/17/2022,25.70,3/21/2022,24.83,3.385214
43,1050.SR,3/22/2022,25.94,3/30/2022,26.61,-2.582884
42,1050.SR,4/25/2022,27.48,4/26/2022,26.33,4.184862
41,1050.SR,5/29/2022,24.49,5/30/2022,26.23,-7.104941
40,1050.SR,6/16/2022,24.08,6/19/2022,22.66,5.897010
39,1050.SR,6/28/2022,23.14,6/30/2022,22.85,1.253241
38,1050.SR,7/18/2022,24.49,7/19/2022,24.78,-1.184157
37,1050.SR,7/21/2022,24.83,7/24/2022,24.54,1.167942


In [55]:
entryexitframe[::-1].to_csv("banquesaudifransientryexit.csv")

In [56]:
from google.colab import drive
drive.mount('drive')
!cp banquesaudifransientryexit.csv "/content/drive/My Drive/"

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).
